# Clean-up your SageMaker environment

❗ This notebook removes all resources created by all notebooks you run in your environment. The following code cells will:
- permanently delete project or projects you provisioned in your Studio environment
- permanently delete feature group or groups
- permanently delete project-provisioned S3 buckets
- permanently delete objects in S3 buckets under project-related prefixes
- permanently delete inference endpoints

<div class="alert alert-info"> 💡 <strong> This notebook will permanently delete resources in your AWS account. Please double check the names of resources to be deleted! </strong>
</div>

In [4]:
import sagemaker
import boto3
import time
import json
import os

In [5]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
abalone_dataset_file_name                -> 'abalone.csv'
abalone_dataset_local_url                -> '../dataset/abalone.csv'
auto_ml_job_name                         -> 'automl-asinproc-24-14-33-56'
bucket_name                              -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                            -> 'from-idea-to-prod/xgboost'
customers_count                          -> 10000
customers_feature_group_name             -> 'fscw-customers-07-20-17-46'
data_bucket                              -> 'sagemaker-us-east-1-906545278380'
data_uploaded                            -> True
domain_id                                -> 'd-r8pbvl3oamh6'
dw_flow_file_url                         -> 's3://sagemaker-us-east-1-906545278380/feature-sto
dw_output_name                           -> '928854ec-259e-4130-8e0f-b65221b27d6e.default'
endpoint_name                            -> 'reorder-classifier-2022-07-20-19-45-09-338'
execution_role      

In [6]:
sm = boto3.client("sagemaker")
s3 = boto3.resource('s3')

## Delete projects

In [7]:
# Get all projects created by the current domain
projects = [
    {"ProjectName":p["ProjectName"], "ProjectId":p["ProjectId"]} for p in sm.list_projects(MaxResults=100, SortBy="CreationTime")["ProjectSummaryList"] 
        if sm.describe_project(ProjectName=p["ProjectName"])["CreatedBy"]["DomainId"] == domain_id and p["ProjectStatus"] == "CreateCompleted"
]

print(f"These projects have been created by domain {domain_id}: {json.dumps(projects, indent=2)}")

These projects have been created by domain d-r8pbvl3oamh6: [
  {
    "ProjectName": "step5-deploy-model",
    "ProjectId": "p-yh0gano2qax2"
  },
  {
    "ProjectName": "step4-build-model",
    "ProjectId": "p-82bb0rqxd46j"
  },
  {
    "ProjectName": "s3-fs-ingest-06-19-30-11",
    "ProjectId": "p-iwfgodxukuoo"
  },
  {
    "ProjectName": "MLOps-demo-2022",
    "ProjectId": "p-mf8uanujx8tn"
  }
]


In [ ]:
# Select projects to be deleted
projects_to_delete = []

for p in projects:
    print(f"Are you sure you want to delete this project: {p['ProjectName']}? (y/n)")
    choice = input()
    if choice == 'y':
        projects_to_delete.append(p)
        
print(f"***************************************")
print(f"The following projects will be deleted:\n{json.dumps(projects_to_delete, indent=2)}")
print(f"***************************************")

In [17]:
for p in projects_to_delete:
    try:
        print(f"Deleting project {p['ProjectName']}:{sm.delete_project(ProjectName=p['ProjectName'])}")
    except Exception:
        pass

## Delete CloudFormation stacks
This section deletes the AWS CloudFormation stacks that projects created

In [25]:
cfn = boto3.client("cloudformation")

for p in projects_to_delete:
    for s in [
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-staging",
            f"sagemaker-{p['ProjectName']}-{p['ProjectId']}-deploy-prod"
            ]:
        try:
            print(f"Delete CloudFormation stack: {s}")
            r = cfn.delete_stack(StackName=s)
            print(r)
            time.sleep(180)
        except Exception:
            print(f"Exception in deleting {s}")
            pass

## Remove inference endpoints

## Delete feature groups

In [8]:
feature_groups = sm.list_feature_groups(
    FeatureGroupStatusEquals="Created", 
    SortOrder="Descending", 
    SortBy="CreationTime"
)["FeatureGroupSummaries"]

In [9]:
feature_groups

[{'FeatureGroupName': 'fscw-orders-07-20-17-46',
  'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:906545278380:feature-group/fscw-orders-07-20-17-46',
  'CreationTime': datetime.datetime(2022, 7, 20, 17, 54, 12, 176000, tzinfo=tzlocal()),
  'FeatureGroupStatus': 'Created',
  'OfflineStoreStatus': {'Status': 'Active'}},
 {'FeatureGroupName': 'fscw-products-07-20-17-46',
  'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:906545278380:feature-group/fscw-products-07-20-17-46',
  'CreationTime': datetime.datetime(2022, 7, 20, 17, 47, 10, 427000, tzinfo=tzlocal()),
  'FeatureGroupStatus': 'Created',
  'OfflineStoreStatus': {'Status': 'Active'}},
 {'FeatureGroupName': 'fscw-customers-07-20-17-46',
  'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:906545278380:feature-group/fscw-customers-07-20-17-46',
  'CreationTime': datetime.datetime(2022, 7, 20, 17, 46, 38, 715000, tzinfo=tzlocal()),
  'FeatureGroupStatus': 'Created',
  'OfflineStoreStatus': {'Status': 'Active'}},
 {'FeatureGroupName': 't

In [ ]:
# Select feature groups to be deleted
feature_groups_to_delete = []

for fg in feature_groups:
    print(f"Are you sure you want to delete this feature group: {fg['FeatureGroupName']}? (y/n)")
    choice = input()
    if choice == 'y':
        feature_groups_to_delete.append(fg["FeatureGroupName"])
        
print(f"********************************************")
print(f"The following feature groups will be deleted:\n{json.dumps(feature_groups_to_delete, indent=2)}")
print(f"********************************************")

In [11]:
def delete_offline_store(feature_group_name: str):
    try:
        offline_store_config = sm.describe_feature_group(FeatureGroupName=feature_group_name)['OfflineStoreConfig']

    except Exception:
        print(f'Feature group: {feature_group_name} does NOT have an offline store!')
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['ResolvedOutputS3Uri']
    print(f"all feature store objects under {offline_store_s3_uri} will be deleted!")
    print("Are you sure you want to these objects ? (y/n)")
    
    choice = input()
    if choice == 'y':
        !aws s3 rm {offline_store_s3_uri} --recursive

<div class="alert alert-info"> 💡 <strong> The following code cell will delete the selected feature groups!</strong>
</div>

In [12]:
for fg in feature_groups_to_delete:
    print(f"Deleting the feature group: {fg}")
    delete_offline_store(fg)
    sm.delete_feature_group(FeatureGroupName=fg)

## Delete project-provisioned S3 buckets

<div class="alert alert-info"> 💡 <strong> The following code cell will delete all S3 buckets created by a project!</strong>
</div>

In [18]:
print(f"*****************************************************")
print(f"The following S3 buckets will be removed permanently!")
print(f"*****************************************************")
for p in projects_to_delete:
    print(f"sagemaker-project-{p['ProjectId']}")

*****************************************************
The following S3 buckets will be removed permanently!
*****************************************************


In [19]:
for p in projects_to_delete:
    !aws s3 rb s3://sagemaker-project-{p['ProjectId']} --force 

## Remove project-related objects from S3 data bucket

<div class="alert alert-info"> 💡 <strong> The following code cells will delete all objects under specified S3 prefixes!</strong>
</div>

In [21]:
prefixes_to_delete = [
    bucket_prefix,
    #s3_fs_query_output_prefix
]

In [23]:
print(f"************************************************************************")
print(f"All objects under the following S3 prefixes will be removed permanently!")
print(f"************************************************************************")
for p in prefixes_to_delete:
    print(f"{bucket_name}/{p}")

************************************************************************
All objects under the following S3 prefixes will be removed permanently!
************************************************************************
sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost


<div class="alert alert-info"> ❗ <strong> Double check the S3 prefix! All data under this prefix will be permanently deleted!</strong>
</div>

In [ ]:
# Un-comment the rm command
for p in prefixes_to_delete:
#   !aws s3 rm s3://{bucket_name}/{p} --recursive

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>